<a href="https://colab.research.google.com/github/benevolent-machines/crm-super-deduper/blob/main/main-module.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CRM Super Deduper Module

https://github.com/benevolent-machines/crm-super-deduper


## _load_data()

In [ ]:
def _load_data(input_data):
    import pandas as pd

    if isinstance(input_data, str):
        file_extension = input_data.split('.')[-1].lower()
        if file_extension == 'csv':
            df = pd.read_csv(input_data)
        elif file_extension in ['xlsx', 'xls']:
            df = pd.read_excel(input_data)
        else:
            raise ValueError(
                f"load_data(): Unsupported file type: {file_extension}")
    elif isinstance(input_data, pd.DataFrame):
        df = input_data
    else:
        raise ValueError(
            f"load_data(): Unsupported file type: {file_extension}")
    return df
#_load_data("test.csv")

## _tfidf_neighbors()

In [ ]:
def _tfidf_neighbors(
        pandas_dataframe,
        entity_id='entity_id',
        columns=['name', 'address', 'city', 'state'],
        threshold=0.7
    ):

    import numpy as np
    import pandas as pd
    from sklearn.feature_extraction.text import TfidfVectorizer
    from sklearn.neighbors import NearestNeighbors

    df = pandas_dataframe
    print(df.shape)

    # descending by entity_id so we start with the latest records
    df = df.sort_values(entity_id, ascending=False).reset_index(drop=True)

    df[columns] = df[columns].astype(str) # clean up NaN
    df['combined'] = df[columns].agg(' '.join, axis=1) # TF/IDF documents
    df['combined'] = df['combined'].str.upper()
    df['dup_id'] = df[entity_id]
    df['dup_flag'] = 0
    df['score'] = 0.0

    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(df['combined'])
    nbrs = NearestNeighbors(radius=threshold, metric='cosine').fit(tfidf_matrix)
    distances, indices = nbrs.radius_neighbors(tfidf_matrix)

    for i, (dist, idx) in enumerate(zip(distances, indices)):
        for d, j in zip(dist, idx):
            if i < j:  # only compare with what comes after:
                #print('distance',d)
                similarity_score = 1 - d  # Convert distance to similarity
                if similarity_score > threshold:
                    if df.loc[i, 'dup_flag'] == 0:
                        df.loc[i, 'dup_flag'] = 2
                    df.loc[j, 'dup_flag'] = 1
                    if df.loc[j, 'dup_id'] == df.loc[j, entity_id]:
                        df.loc[j, 'dup_id'] = df.loc[i, entity_id]
                        if df.loc[i, 'score'] == 0.0:
                            df.loc[i, 'score'] = 1.0
                        df.loc[j, 'score'] = round(similarity_score,3)
    return df

## _tfidf_neighbors_subgrouped()

In [ ]:
def _tfidf_neighbors_subgrouped(
        pandas_dataframe,
        entity_id='entity_id',
        columns=['name', 'address', 'city', 'state'],
        subgroup_column='state',  # Specify the column to use for subgrouping
        threshold=0.7
    ):
    import numpy as np
    import pandas as pd
    from sklearn.feature_extraction.text import TfidfVectorizer
    from sklearn.neighbors import NearestNeighbors

    df = pandas_dataframe

    # Ensure the subgroup column is a string
    df[subgroup_column] = df[subgroup_column].astype(str)

    # Create a subgroup identifier based on the first 2 characters of the specified column
    df['subgroup'] = df[subgroup_column].str[:2].str.upper()

    # Initialize a list to collect dataframes
    dfs = []

    for subgroup, subgroup_df in df.groupby('subgroup'):
        print(f"subgroup: {subgroup} shape[0]: {subgroup_df.shape[0]}")
        subgroup_df = subgroup_df.sort_values(entity_id, ascending=False).reset_index(drop=True)
        subgroup_df[columns] = subgroup_df[columns].astype(str)  # Clean up NaN
        subgroup_df['combined'] = subgroup_df[columns].agg(' '.join, axis=1)  # TF/IDF documents
        subgroup_df['combined'] = subgroup_df['combined'].str.upper()
        subgroup_df['dup_id'] = subgroup_df[entity_id]
        subgroup_df['dup_flag'] = 0
        subgroup_df['score'] = 0.0

        vectorizer = TfidfVectorizer()
        tfidf_matrix = vectorizer.fit_transform(subgroup_df['combined'])
        nbrs = NearestNeighbors(radius=threshold, metric='cosine').fit(tfidf_matrix)
        distances, indices = nbrs.radius_neighbors(tfidf_matrix)

        for i, (dist, idx) in enumerate(zip(distances, indices)):
            for d, j in zip(dist, idx):
                if i < j:  # only compare with what comes after
                    similarity_score = 1 - d  # Convert distance to similarity
                    if similarity_score > threshold:
                        if subgroup_df.loc[i, 'dup_flag'] == 0:
                            subgroup_df.loc[i, 'dup_flag'] = 2
                        subgroup_df.loc[j, 'dup_flag'] = 1
                        if subgroup_df.loc[j, 'dup_id'] == subgroup_df.loc[j, entity_id]:
                            subgroup_df.loc[j, 'dup_id'] = subgroup_df.loc[i, entity_id]
                            if subgroup_df.loc[i, 'score'] == 0.0:
                                subgroup_df.loc[i, 'score'] = 1.0
                            subgroup_df.loc[j, 'score'] = round(similarity_score, 3)

        # Add processed subgroup to the list
        dfs.append(subgroup_df)

    # Concatenate all dataframes in the list at once
    results_df = pd.concat(dfs, ignore_index=True)

    # Drop the temporary 'subgroup' column from the results dataframe
    results_df.drop(columns=['subgroup'], inplace=True)

    return results_df

## _crm_templates

In [ ]:
_crm_templates = {
    "donorperfect": {
        "record": "https://www.donorperfect.net/prod/donor.asp"
            + "?donor_id={entity_id}",
        "merge": "https://www.donorperfect.net/prod/combinedonorsStart.asp"
            + "?donor_id1={entity_id}&donor_id2={dup_id}&combineType=Advanced"
    },
}

## _excel_redirect()

In [ ]:
def _excel_redirect(url):
    redirector = "https://benevolentmachines.org/excel_redirect.html"
    url = redirector + '?page=' + url.replace('?',"&")
    return url

## _output_excel()

In [ ]:
#test
!pip install XlsxWriter

In [ ]:
def _output_excel(
    pandas_dataframe,
    entity_id = 'entity_id',
    compare = ['name', 'address', 'city', 'state'],
    crm = None,
):
    import pandas as pd
    import xlsxwriter

    # organize the columns
    initial = ['dup_flag', entity_id, 'dup_id', 'score']
    reference = []  # additional fields
    xdf = pandas_dataframe
    for col in xdf.columns: # extract references for the spreadsheet
        if col not in initial and col not in compare:
            if col != 'rownum' and col != 'combined':
                reference.append(col)
    ordered = initial + compare + reference
    if crm is not None:
        crm = crm.lower()

    # prepare the dataframe for the spreadsheet
    xdf = xdf[xdf['dup_flag'] > 0] # the spreadsheet only includes duplicates
    xdf = xdf[ordered] # the order that will appear in the spreadsheet
    xdf = xdf.sort_values(['dup_id','dup_flag'], ascending=[False, False])
    xdf = xdf.reset_index(drop=True) # renumber after sorting
    xdf = xdf.fillna('') # important
    xdf.rename(columns={'dup_flag': 'type'}, inplace=True)
    xdf['type'] = xdf['type'].map({2: 'last', 1: 'prior'}).astype(str)
    crm_flag = False
    if crm is not None:
        crm_flag = True
        xdf.insert(loc=3, column='action', value='')

    excel_file = 'near_duplicates.xlsx'
    writer = pd.ExcelWriter(excel_file, engine='xlsxwriter')

    xdf.to_excel(writer, index=False, sheet_name='Sheet1', startrow=1, header=False)
    workbook = writer.book
    worksheet = writer.sheets['Sheet1']

    # Custom formats
    header_grey_left = workbook.add_format(
        {'bg_color': '#D3D3D3', 'bold': True, 'align': 'left'})
    header_grey_right = workbook.add_format(
        {'bg_color': '#D3D3D3', 'bold': True, 'align': 'right'})
    header_green_left = workbook.add_format(
        {'bg_color': '#B8D7A3', 'bold': True, 'align': 'left'})

    last_blue_left = workbook.add_format(
        {'bg_color': '#ADD8E6', 'align': 'left'})
    last_blue_right = workbook.add_format(
        {'bg_color': '#ADD8E6', 'align': 'right'})
    last_blue_right_score = workbook.add_format(
        {'bg_color': '#ADD8E6', 'align': 'right', 'num_format': '0.000'})
    last_blue_right_date = workbook.add_format(
        {'bg_color': '#ADD8E6', 'align': 'right', 'num_format': 'yyyy-mm-dd'})
    last_blue_right_dec2 = workbook.add_format(
        {'bg_color': '#ADD8E6', 'align': 'right', 'num_format': '#,##0.00'})
    last_blue_right_dec3 = workbook.add_format(
        {'bg_color': '#ADD8E6', 'align': 'right', 'num_format': '##0.000'})
    last_blue_left_link = workbook.add_format(
        {'bg_color': '#ADD8E6', 'align': 'left',
        'font_color': 'blue', 'underline': 1})
    last_blue_right_link = workbook.add_format(
        {'bg_color': '#ADD8E6', 'align': 'right',
        'font_color': 'blue', 'underline': 1})

    prior_left = workbook.add_format({'align': 'left'})
    prior_right = workbook.add_format({'align': 'right'})
    prior_right_score = workbook.add_format(
        {'align': 'right', 'num_format': '0.000'})
    prior_right_date = workbook.add_format(
        {'align': 'right', 'num_format': 'yyyy-mm-dd'})
    prior_right_dec2 = workbook.add_format(
        {'align': 'right', 'num_format': '#,##0.00'})
    prior_right_dec3 = workbook.add_format(
        {'align': 'right', 'num_format': '##0.000'}) # score
    prior_left_link = workbook.add_format(
        {'align': 'left', 'font_color': 'blue', 'underline': 1})
    prior_right_link = workbook.add_format(
        {'align': 'right', 'font_color': 'blue', 'underline': 1})

    ##### determine column properties
    tdf = xdf[0:500] # look at first n rows to extract properties
    col_info = {} # info about the columns
    for col in tdf.columns:
        col_props = {}
        is_numeric = is_string = is_date = False
        max_decimals = 0
        is_numeric = pd.to_numeric(
            tdf[col].dropna(), errors='coerce').notnull().all()
        if is_numeric:
            numeric_series = pd.to_numeric(tdf[col], errors='coerce').dropna()
            decimal_lengths = numeric_series.apply(
                lambda x: len(str(x).split('.')[1]) if '.' in str(x) else 0)
            max_decimals = decimal_lengths.max()
        else:
            is_date = pd.to_datetime(
                tdf[col].dropna(), errors='coerce').notnull().all()
            if not is_date:
                is_string = tdf[col].dtype == 'object'
        col_props['is_numeric'] = is_numeric
        col_props['is_string'] = is_string
        col_props['is_date'] = is_date
        col_props['max_decimals'] = max_decimals
        col_info[col] = col_props

    # column settings
    for num, col in enumerate(tdf.columns):

        # column widths
        max_len = max(tdf[col].astype(str).map(len).max(), len(col)) + 1
        if max_len > 15:
            max_len = 15
        worksheet.set_column(num, num, max_len)

        # custom headers
        if col in compare: # comparison columns get a green header
            worksheet.write(0, num, col, header_green_left)
        else:
            if col == 'type':
                worksheet.write(0, num, col, header_grey_right)
            elif col_info[col]['is_string']:
                worksheet.write(0, num, col, header_grey_left)
            else:
                worksheet.write(0, num, col, header_grey_right)

    # cell settings
    for idx, row in xdf.iterrows():
        row_num = idx + 1  # account for header row
        for col_num, col in enumerate(tdf.columns):
            if row['type'] == 'last':
                if col == 'type':
                    worksheet.write(row_num, col_num, row[col], last_blue_right)
                elif col == entity_id:
                    if crm_flag is True: # hyperlinks
                        url = _crm_templates[crm]["record"].format(
                            entity_id = row[entity_id]
                        )
                        url = _excel_redirect(url) # excel nonsense
                        if col_info[col]["is_string"]:
                            worksheet.write_url(row_num, col_num, url,
                                last_blue_left_link,
                                string=str(row[entity_id]))
                        else:
                            worksheet.write_url(row_num, col_num, url,
                                last_blue_right_link,
                                string=str(row[entity_id]))
                    else:
                        if col_info[col]["is_string"]:
                            worksheet.write(
                                row_num, col_num, row[col], last_blue_left)
                        else:
                            worksheet.write(
                                row_num, col_num, row[col], last_blue_right)
                elif col == 'action': # blank and blue
                    worksheet.write(row_num, col_num, '', last_blue_right)
                elif col == 'score': # blank and blue
                    worksheet.write(row_num, col_num, '', last_blue_right)
                else:
                    if col_info[col]["is_string"]:
                        worksheet.write(row_num, col_num, row[col], last_blue_left)
                    elif col_info[col]['is_numeric']:
                        if col_info[col]['max_decimals'] == 2:
                            worksheet.write(row_num, col_num, row[col], last_blue_right_dec2)
                        else:
                            worksheet.write(row_num, col_num, row[col], last_blue_right)
                    elif col_info[col]['is_date']:
                        worksheet.write(row_num, col_num, row[col], last_blue_right_date)
                    else: # other?
                        worksheet.write(row_num, col_num, row[col], last_blue_right)
            else: # prior
                if col == 'type':
                    worksheet.write(row_num, col_num, row[col], prior_right)
                elif col == entity_id:
                    if crm_flag is True: # hyperlinks
                        url = _crm_templates[crm]["record"].format(
                            entity_id = row[entity_id]
                        )
                        url = _excel_redirect(url) # excel nonsense
                        if col_info[col]["is_string"]:
                            worksheet.write_url(row_num, col_num, url,
                                prior_left_link, string=str(row[entity_id]))
                        else:
                            worksheet.write_url(row_num, col_num, url,
                                prior_right_link, string=str(row[entity_id]))
                    else:
                        if col_info[col]["is_string"]:
                            worksheet.write(
                                row_num, col_num, row[col], prior_left)
                        else:
                            worksheet.write(
                                row_num, col_num, row[col], prior_right)
                elif col == 'action':
                    if crm_flag is True:
                        url = _crm_templates[crm]["merge"].format(
                            entity_id = row[entity_id],
                            dup_id = row["dup_id"]
                        )
                        url = _excel_redirect(url) # excel nonsense
                        worksheet.write_url(row_num, col_num, url,
                            prior_left_link, string='merge')
                elif col == 'score':
                    worksheet.write(row_num, col_num, row[col], prior_right_dec3)
                else:
                    if col_info[col]["is_string"]:
                        #print(col)
                        worksheet.write(row_num, col_num, row[col], prior_left)
                    elif col_info[col]['is_numeric']:
                        if col_info[col]['max_decimals'] == 2:
                            worksheet.write(row_num, col_num, row[col], prior_right_dec2)
                        else:
                            worksheet.write(row_num, col_num, row[col], prior_right)
                    elif col_info[col]['is_date']:
                        worksheet.write(row_num, col_num, row[col], prior_right_date)
                    else: # other?
                        worksheet.write(row_num, col_num, row[col], prior_right)

    worksheet.freeze_panes(1, 4)
    writer.close()
    return xdf

## run()

In [ ]:
def run(
    input_data, # file (xls,xlsx,csv) or dataframe
    entity_id='entity_id',
    compare = ['name', 'address', 'city', 'state'],
    crm_links = None,
    score_minimum = .67,
    filter_column = None, # 'gift_total'
    filter_minimum = None, # 2_000
):
    print("loading data")
    df = _load_data(input_data) # load file
    #df = df[0:10000]
    print("processing algorithms")
    df = _tfidf_neighbors(df, entity_id, compare) # TF/IDF with neighbor cosines

    if filter_column is not None and filter_minimum is not None:
        # filter on duplications involving filter_field at a filter_minimum
        dup_ids = list(
            df[df[filter_column] >= filter_minimum]['dup_id'].unique())
        df = df[df['dup_id'].isin(dup_ids)]

    print("outputting results")
    _output_excel(df, entity_id, compare, crm_links)
    return


# TESTS

## CRM Export File

In [ ]:
#test
run(
    input_data = "donors.csv", # exported from the CRM
    entity_id = "donor_id", # unique id
    compare = ['first_name', 'last_name', 'address', 'city', 'state'],
    crm = "donorperfect"
)

In [ ]:
#test
gdf = _load_data('donors.csv')
df = _tfidf_neighbors(
    gdf, threshold=.60,
    entity_id='donor_id',
    columns=['first_name', 'last_name', 'address', 'city', 'state'])

In [ ]:
#test
df.shape

In [ ]:
#test

print(df[crit].shape)
df[crit][['donor_id','dup_id','dup_flag','score',
          'first_name', 'last_name', 'address', 'city', 'state','zip','gift_total','last_contrib_date']].sort_values(
              ['dup_id','dup_flag'], ascending=[False, False])

In [ ]:
#test
import pandas as pd
df['prefix'] = df['last_name'].str[:2]
groups_with_diff_city_prefix = df.groupby('dup_id').filter(lambda x: x['prefix'].nunique() > 1)
result = groups_with_diff_city_prefix.sort_values(by=['dup_id', 'prefix'])
display(result[['donor_id','dup_id','dup_flag','score',
          'first_name', 'last_name', 'address', 'city', 'state','zip','gift_total','last_contrib_date']]
        .sort_values(['dup_id','dup_flag'], ascending=False))

## Test Data

### _first_name_variations

In [ ]:
#test
_first_name_variations = {
    'Dave': 'David',
    'Joe': 'Joseph',
    'Elizabeth': 'Betty',
    'Bill': 'William',
    'Bob': 'Robert',
    'Tom': 'Thomas',
    'Dick': 'Richard',
    'Jim': 'James',
    'Jenny': 'Jennifer',
    'Kathy': 'Katherine',
    'Sue': 'Susan',
    'Maggie': 'Margaret',
    'Harry': 'Harold',
    'Ron': 'Ronald',
    'Nick': 'Nicholas',
    'Patty': 'Patricia',
    'Chris': 'Christopher',
    'Mike': 'Michael',
    'Steve': 'Steven',
    'Andy': 'Andrew',
    'Matty': 'Matthew',
    'Sam': 'Samuel',
    'Tony': 'Anthony',
}

### _first_name_variation()

In [ ]:
#test
def _first_name_variation(full_name, chance=40):
    import random
    name_parts = full_name.split()
    first_name = name_parts[0]
    if random.random() < (chance / 100.0):
        if first_name in _first_name_variations:
            first_name = _first_name_variations[first_name]
        else:
            for key, value in _first_name_variations.items():
                if first_name == value:
                    first_name = key
                    break
    return ' '.join([first_name] + name_parts[1:])

#print(_first_name_variation("Joe Satriani",chance=100))
#print(_first_name_variation("David Bowie",chance=100))

### _last_name_transposition()

In [ ]:
#test
def _last_name_transposition(full_name, chance=1):
    import random
    name_parts = full_name.split()
    if len(name_parts) > 1 and random.random() < (chance / 100.0):
        last_name = name_parts[-1]
        if len(last_name) > 4:
            i = random.randint(1, len(last_name) - 2)
            last_name_chars = list(last_name)
            last_name_chars[i], last_name_chars[i+1] = \
              last_name_chars[i+1], last_name_chars[i]
            name_parts[-1] = ''.join(last_name_chars)
    return ' '.join(name_parts)
#print(_last_name_transposition("Jon Anderson", chance=100))

### _generate_test_data()

In [ ]:
#test
!pip install mimesis # used for generating test data

In [ ]:
#test
def _generate_test_data(n):

    from mimesis import Person, Address, Finance
    import pandas as pd
    import random

    data = []
    entity_id = 0

    person = Person('en')
    address_gen = Address('en')
    finance = Finance('en')

    for _ in range(n):
        entity_id += 1
        is_person = True
        name = person.full_name()
        if random.random() < (10.0 / 100.0):
            is_person = False
            name = finance.company()
        street_address = address_gen.address()
        city = address_gen.city()
        state = address_gen.state(abbr=True)
        label = 0  # not a duplicate
        data.append([entity_id, name, street_address, city, state, label])
        if random.random() < (1.0 / 100.0): # exact duplicate
            entity_id += 1
            label = 1
            data[-1][-1] = label # update the prior label
            data.append([entity_id, name, street_address, city, state, label])
        if is_person:
            dup_name = _first_name_variation(name, chance=30)
            dup_name = _last_name_transposition(dup_name, chance=1)
            if name != dup_name: # fuzzy duplicate
                entity_id += 1
                label = 1
                data[-1][-1] = label # update the prior label
                data.append([
                    entity_id, dup_name, street_address, city, state, label])
    random.shuffle(data) # mix them up!
    df = pd.DataFrame(data,
        columns=['entity_id', 'name', 'address', 'city', 'state', 'label'])
    df['entity_id'] = df.index + 1 # renumber to shuffled order
    df.to_csv('test.csv', index=False) # test data for passing to _load_data()

    return df
#_generate_test_data(1000)

In [ ]:
#test
gdf = _generate_test_data(10000)

In [ ]:
#test
df = _tfidf_neighbors_subgrouped(
    gdf, threshold=.60, subgroup_column="city"
    )

In [ ]:
#test
"""
class m: pass
gdf = _generate_test_data(1000)
glabel = gdf['label'] > 0
print('test labels:', gdf[glabel].shape[0])
df = _tfidf_neighbors(gdf, threshold=.75)

#print('tfidf labels:', df[df['label'] > 0].shape[0])
#print('tfidf labels found:', df[(df['label'] > 0) & (df['dup_flag'] > 0)].shape[0])
#print('tfidf found:', df[(df['dup_flag'] > 0)].shape[0])
"""
crit = (df['dup_flag'] > 0) # | (df['label'] > 0)
print(df[crit].shape)
df[crit][['entity_id','dup_id','dup_flag','score','label',
          'name','address','city','state','combined']].sort_values(
              ['dup_id','dup_flag'], ascending=[False, False])

In [ ]:
#test
# prep generated data
compare = ['name', 'address', 'city', 'state']
reference = ['label']
columns_order = ['entity_id', 'dup_id', 'dup_flag', 'score'] + compare + reference
xdf = df[crit][columns_order].sort_values(
              ['dup_id','dup_flag'], ascending=[False, False])
xdf = xdf[0:500].reset_index(drop=True)
xdf.shape[0]

In [ ]:
#test
# ['first_name', 'last_name', 'address', 'city', 'state']
# prep exported  data
crm_flag = True
entity_id = 'donor_id'

compare = ['first_name', 'last_name', 'address', 'city', 'state']
reference = ['gift_total', 'created_by', 'last_contrib_date', 'last_contrib_amt', 'zip']
columns_order = initial + compare + reference
xdf = df[crit][columns_order].sort_values(['dup_id','dup_flag'], ascending=[False, False])
xdf = xdf[0:500].reset_index(drop=True)
xdf.rename(columns={'dup_flag': 'type'}, inplace=True)
xdf['type'] = xdf['type'].map({2: 'last', 1: 'prior'}).astype(str)
if crm_flag is True:
    xdf.insert(loc=3, column='action', value='')
output_columns = xdf.columns.tolist() # direct mapping to output
print(output_columns)
xdf #.shape[0]

## Analyses

### _rows_vs_time_plot()

In [ ]:
#test
def _rows_vs_time_plot():
    import pandas as pd
    import plotly.express as px
    import numpy as np
    cdf = pd.DataFrame()
    cdf['rows'] = list(range(10_000, 1_010_000, 10_000))
    cdf['compares'] = cdf['rows'] * (cdf['rows'] - 1) / 2
    cdf['time'] = cdf['compares'] / 21_766_643  # seconds
    cdf['time_delta'] = pd.to_timedelta(cdf['time'], unit='s')
    cdf['time_seconds'] = cdf['time_delta'].dt.total_seconds()
    max_hours = cdf['time_delta'].dt.total_seconds().max() / 3600
    tick_interval_minutes = 15
    tick_interval = pd.Timedelta(minutes=tick_interval_minutes)
    tick_vals = [i*tick_interval for i in range(int(max_hours*6) + 1)]
    tick_vals_seconds = [tick.total_seconds() for tick in tick_vals]
    tick_labels = [(tick.total_seconds() // 3600,
    (tick.total_seconds() % 3600) // 60) for tick in tick_vals]
    tick_text = [f"{int(h):02d}:{int(m):02d}" for h, m in tick_labels]
    fig = px.line(cdf, x='rows', y='time_seconds', labels={'time_delta': 'Time (hh:mm)'})
    fig.update_traces(textposition='top center')
    fig.update_yaxes(tickvals=tick_vals_seconds, ticktext=tick_text)
    fig.update_xaxes(tickvals=list(range(50_000, 1_010_000, 50_000)), tickangle=-90)
    fig.update_layout(yaxis_title="Time (hh:mm)", xaxis_title="Number of Rows")
    fig.show()
_rows_vs_time_plot()